In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Colab에 Mecab 설치

!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh


Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-06-03 04:54:22--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c2:513, 2406:da00:ff00::22c3:9b0a, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=ZyqYClfyxKG7sWRUtnTo

In [13]:
#맞춤법 검사기(띄어쓰기 포함)
!pip install git+https://github.com/ssut/py-hanspell.git
#normalization
!pip install soynlp
!pip install soylemma

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-kc8is7hg
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-kc8is7hg
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp37-none-any.whl size=4871 sha256=834d4c61ddeb12b94ebbe52ad4a9f496d7a8928447566d56460c43c5e0e7aaf3
  Stored in directory: /tmp/pip-ephem-wheel-cache-1swtyaum/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


In [14]:
import os
import sys
import re

import numpy as np
import pandas as pd

import json

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='white', context='notebook', palette='deep')

from collections import Counter

In [15]:
#data = pd.read_excel('/content/drive/MyDrive/Project/[SJU]창의학기제/lecture_evaluation_2021/data/testset.xlsx')
data = pd.read_csv("/content/drive/MyDrive/Project/[SJU]창의학기제/lecture_evaluation_2021/data/review_datal_all.csv")
data.head()

,url_id,index,lecture_name,professor_name,rate,semester,like,text
0,535020,0,법학입문,이재교,100,20,0,판례평석 잘하면 성적잘 주시는것 같음. 중간 개망치고 기말도 틀린 문제 몇 개 있었...
1,535020,1,법학입문,이재교,80,20,0,과제들 되게 귀찮고 많다 시험은 말빨 좋으면 된다 난 시험은 망쳤지만 과제는 열심히...
2,535020,2,법학입문,이재교,80,20,0,수업은 ppt 띄워놓으시고 하시는데 보통 교안에 없는 다른 관련 사례를 중심으로 많...
3,535020,3,법학입문,이재교,100,20,0,"교재에 없는 내용 많이 얘기해주시고, 말씀하신 내용에서 시험문제 내셔서 교재 그렇게..."
4,535020,4,법학입문,이재교,100,20,0,사이버강의라 괜찮았지만 과제는 꽤 많았어요 시험은 수업 잘 들으면 잘 나옴


In [17]:
print("data 결측값 개수 :",data["text"].isnull().sum())
print("data개수:",len(data))

data 결측값 개수 : 0
data개수: 6653


- 한글만 남기기
- 띄어쓰기와 맞춤법 확인 : hanspell(spell_checker)
- 토큰화
- 불용어(형태소)

In [18]:
stopwords = '이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 고 년 가 한 지 대하 오 말 일 그렇 위하 은 는 함 음 심 습니다 아요 세요 에요 었 였 에 을 를'.split() 

In [19]:
def data_text_preprocessing(data):
  #한글
  import re
  korean = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣]"," ",data) 

  #띄어쓰기, 맞춤법
  #from hanspell import spell_checker
  #spelled_sent = spell_checker.check(korean)
  #hanspell_sent = spelled_sent.checked

  #토큰화와 불용어
  from konlpy.tag import Mecab
  mecab = Mecab() 
  tokenization = mecab.morphs(korean)
  no_stopwords = [token for token in tokenization if token not in stopwords]

  #정규화(ㅋㅋㅋㅋㅋㅋㅋㅋ->ㅋㅋ)
  #from soynlp.normalizer import *
  #normalization = emoticon_normalize(tokenization, num_repeats=2)
  #print(normalization)
    
  return ' '.join(no_stopwords)

In [20]:
#멀티프로세싱으로 시간 단축
from multiprocessing import Pool
 
def use_multiprocess(func, iter, workers):
    pool = Pool(processes=workers)
    result = pool.map(func, iter)
    pool.close()
    return result

In [22]:
#전처리완료
import time
start_time = time.time()
preprocessed = use_multiprocess(data_text_preprocessing, data["text"], 3)
print('실행 시간 :', (time.time() - start_time))

실행 시간 : 10.024641990661621


In [23]:
data["token"] = preprocessed

In [24]:
preprocessed

['판례 평석 잘 면 성적 잘 시 중간 개 망치 기말 도 틀린 문제 몇 개 는데 판례 평석 과제 평가 전부 성적 도 결국 나옴',
 '과제 되게 귀찮 많 다 시험 빨 좋 으면 된다 난 시험 망쳤 지만 과제 열심히 해서 점수 잘 받 았',
 '수업 띄워 놓 으시 시 는데 보통 교안 다른 관련 사례 중심 으로 많이 얘기 수업 만 잘 들으면 시험 도 잘 볼 성적 도 후하 게 시 편 다만 과제 분량 채워 쓰 기 쉽 지만은',
 '교재 내용 많이 얘기 해 시 말씀 신 내용 에서 시험 문제 내 셔서 교재 그렇게 크 게 필요 과제 좀 시간 오래 걸리 기 한데 판례 평석 민사 소송법 강의 에서 도 내주 셔서 어차피 내년 도 셔야 니 계속 해야 거 연습 한다 셈 치 해 시 길 점수 나름 잘 시 방청 도 저 재밌 저 상당히 만족 했 던 강의 강의 내용 도 재밌 시험 문제 도 괜찮 았 구요 학점 원래 잘 시 교수 님 시 니 법학 과 들어와서 다른 강의 듣 기 전 수강 기 엔 최적 화 된 과목 인 듯',
 '사이버 강의 라 괜찮 았 지만 과제 꽤 많 았 어요 시험 수업 잘 들으면 잘 나옴',
 '과제 힘 듦 수업 집중 면 재미 으나 매우 졸림',
 '교수 님 진짜 좋 으 재밌 게 수업 려고 노력 시 이름 도 외우 시 려고 노력 시 게 눈 보임 다만 재미 시 종이 칠 쯤 면 앞 애 다 졸 평석 이나 방청 과제 글 쓰 기 양 늘려서 잘 쓰 에게 문제 성의 만 으면 이쁠 시 수업 시간 사례 위주 로 공부 면 성적 나쁘 게 나옴',
 '교재 정말 필요 당연히 시험 문제 도 그닥 영향 안 줌 교수 님 수업 전 따로 호명 시 면서 눈 마주 치심 수업 내용 좋 은데 과제 많이 빡 셈 내용 빡세 다기 보다 분량 채우 기 힘든 게 큼',
 '일단 법 학부 수업 중 재밌 그리고 열공 한다면 성적 또한 뒷 따름 분명 합니다',
 '교재 꼼꼼히 안 읽 어도 교수 님 강의 녹음 해놨 다가 시험 기간 전 번 어 기 만 면 시험 어렵 게 풀 문제 대부분 직접 평결 내리 형식 임 학기 과제 네 번 는데 다 장

In [25]:
data.to_csv("/content/drive/MyDrive/Project/[SJU]창의학기제/lecture_evaluation_2021/data/data_all_token.csv")